# EXCHANGE_RATE

## Overview
This function retrieves the exchange rate from a base currency to a target currency using a public exchange rate API.  Supports all 161 commonly circulating world currencies, as listed in the [ExchangeRate-API Supported Currencies documentation](https://www.exchangerate-api.com/docs/supported-currencies), e.g. USD, EUR, GBP, JPY, SGD, HKD, MYR, CNY, AUD, CAD, INR, and many more.  See [API Documentation](https://www.exchangerate-api.com/docs/overview) for details.

## Usage
To use the `EXCHANGE_RATE` function in Excel, enter it as a formula in a cell, specifying the base and target currencies:

```excel
=EXCHANGE_RATE(base_currency, target_currency)
```
- `base_currency` and `target_currency` must be single currency codes (e.g., "USD", "SGD").

## Arguments
| Argument       | Type   | Required | Description                                   | Example |
|:--------------|:-------|:---------|:----------------------------------------------|:--------|
| base_currency | string | Yes      | The base currency code (e.g., "USD").        | "USD"   |
| target_currency| string | Yes      | The target currency code (e.g., "SGD").      | "SGD"   |

## Returns
| Returns   | Type   | Description                                  | Example |
|:----------|:-------|:---------------------------------------------|:--------|
| Rate      | float  | The exchange rate from base to target.        | 1.345   |
| Error     | string | Error message if input or API call fails.     | "Error: Invalid currency code" |

## Examples

### USD to SGD
- **Input:**
    - base_currency: "USD"
    - target_currency: "SGD"
- **Excel Formula:**
    ```excel
    =EXCHANGE_RATE("USD", "SGD")
    ```
- **Output:**
    - 1.345 (actual value will vary)

### EUR to USD
- **Input:**
    - base_currency: "EUR"
    - target_currency: "USD"
- **Excel Formula:**
    ```excel
    =EXCHANGE_RATE("EUR", "USD")
    ```
- **Output:**
    - 1.085 (actual value will vary)

In [ ]:
import requests

def exchange_rate(base_currency = "EUR", target_currency = "USD"):
    """
    Get exchange rate from a base currency to a single target currency.
    Only accepts string inputs for both base and target currency codes.

    Args:
        base_currency (str): Base currency code (ISO 4217, e.g., EUR, USD, GBP, JPY, etc.)
        target_currency (str): Target currency code (ISO 4217)

    Returns:
        float: Exchange rate as a single value, or error message as string
    """
    try:
        url = f"https://api.exchangerate-api.com/v4/latest/{base_currency}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if target_currency in data['rates']:
                return data['rates'][target_currency]
            else:
                return f"Target currency '{target_currency}' not found in API response. See https://www.exchangerate-api.com/docs/supported-currencies."
        else:
            return f"Error fetching exchange rates: HTTP {response.status_code}"
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
import ipytest
ipytest.autoconfig()

def test_usd_to_sgd():
    result = exchange_rate("USD", "SGD")
    assert isinstance(result, (float, str))

def test_eur_to_usd():
    result = exchange_rate("EUR", "USD")
    assert isinstance(result, (float, str))

def test_unsupported_base():
    result = exchange_rate("ABC", "USD")
    assert isinstance(result, str)
    assert ("Error" in result or "not found" in result or "HTTP" in result)

def test_unsupported_target():
    result = exchange_rate("USD", "XYZ")
    assert isinstance(result, str)
    assert ("Error" in result or "not found" in result or "HTTP" in result)

ipytest.run()

In [ ]:
# Demo Examples
import gradio as gr

examples = [
    ["USD", "SGD"],
    ["EUR", "USD"],
    ["SGD", "MYR"]
]

demo = gr.Interface(
    fn=exchange_rate,
    inputs=[
        gr.Textbox(label="Base Currency (e.g., USD)", value="USD"),
        gr.Textbox(label="Target Currency (e.g., SGD)", value="SGD"),
    ],
    outputs=gr.Textbox(label="Exchange Rate or Error Message"),
    examples=examples,
    description="Retrieve the exchange rate from a base currency to a target currency using a public API.",
    flagging_mode="never",
)
demo.launch()